In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as T
import torch.optim as optim
import torchvision.models as models
import math
import torch.nn.functional as F
from collections import OrderedDict

In [ ]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module("norm1", nn.BatchNorm2d(num_input_features)),
        self.add_module("relu1", nn.ReLU(inplace=True)),
        self.add_module(
            "conv1",
            nn.Conv2d(
                num_input_features,
                bn_size * growth_rate,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        ),
        self.add_module("norm2", nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module("relu2", nn.ReLU(inplace=True)),
        self.add_module(
            "conv2",
            nn.Conv2d(
                bn_size * growth_rate,
                growth_rate,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
        ),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(
                new_features, p=self.drop_rate, training=self.training
            )
        return torch.cat([x, new_features], 1)

In [ ]:
class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate
            )
            self.add_module("denselayer%d" % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module("norm", nn.BatchNorm2d(num_input_features))
        self.add_module("relu", nn.ReLU(inplace=True))
        self.add_module(
            "conv",
            nn.Conv2d(
                num_input_features,
                num_output_features,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        )
        self.add_module("pool", nn.AvgPool2d(kernel_size=2, stride=2))

In [ ]:
class DenseNet121(nn.Module):

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64, bn_size=4, drop_rate=0,num_classes=10,):
        super(DenseNet121, self).__init__()
        # First convolution
        # CIFAR-10: kernel_size 7 ->3, stride 2->1, padding 3->1
        self.features = nn.Sequential(
            OrderedDict(
                [ ("conv0", nn.Conv2d(3, num_init_features, kernel_size=3, stride=1, padding=1, bias=False,),),
                  ("norm0", nn.BatchNorm2d(num_init_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
                  ("relu0", nn.ReLU(inplace=True)),
                  ("pool0", nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)),
                ]
            )
        )

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
            )
            self.features.add_module("denseblock%d" % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(
                    num_input_features=num_features,
                    num_output_features=num_features // 2,
                )
                self.features.add_module("transition%d" % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module("norm5", nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out


In [ ]:
transforms = T.Compose([
                        T.RandomCrop((32,32),padding=(4,4)),
                        T.RandomHorizontalFlip(p=0.5),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                        
])

trainset = dsets.CIFAR10(root = '/content/drive/MyDrive/lab/InceptionV2/datasets',train=True,transform=transforms,download=True)
testset = dsets.CIFAR10(root='/content/drive/MyDrive/lab/InceptionV2/datasets',train=False,transform=T.Compose([         #T.RandomCrop((32,32),padding=(4,4)),
                                                                                                                    # T.RandomHorizontalFlip(p=0.5),
                                                                                                                    T.ToTensor(),
                                                                                                                    T.Normalize(mean=[0.485,0.456,0.406],
                                                                                                                                std = [0.229,0.224,0.225])
]),download=True)
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


Extracting /content/drive/MyDrive/lab/InceptionV2/datasets/cifar-10-python.tar.gz to /content/drive/MyDrive/lab/InceptionV2/datasets
Files already downloaded and verified


In [ ]:
trainLoader = torch.utils.data.DataLoader(trainset,batch_size = 128,shuffle = True)
testLoader = torch.utils.data.DataLoader(testset,batch_size =128,shuffle=False,drop_last=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = DenseNet121(num_classes=100)
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [ ]:
epochs_accuracy_list = []
result_train_avgtop1acc = []
result_train_avgtop5acc = []
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')
epochs_accuracy = 0
lr = 0.05
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0

    if (epoch==49):
      optimizer = optim.SGD(net.parameters(),lr=0.01,momentum=0.9,weight_decay=5e-4)
    elif (epoch==74):
      optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9,weight_decay=5e-4)

    for i, data in enumerate(trainLoader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # print(inputs.shape)  
        outputs= net(inputs)

        acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
        top1.update(acc1[0], inputs.size(0))
        top5.update(acc5[0], inputs.size(0))

        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    
    acc = check_accuracy(net,testLoader)
    print("epoch: {} | Accuracy: {}".format(epoch+1 ,acc))
    print('====> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}' .format(top1=top1, top5=top5))
    result_train_avgtop1acc.append(top1.avg)
    result_train_avgtop5acc.append(top5.avg)

print('Finished Training')

[1,   100] loss: 0.465
[1,   200] loss: 0.480
[1,   300] loss: 0.479
epoch: 1 | Accuracy: 78.59059293316025
====> Acc@1 83.594 Acc@5 99.278
[2,   100] loss: 0.464
[2,   200] loss: 0.481
[2,   300] loss: 0.464
epoch: 2 | Accuracy: 82.08845457572474
====> Acc@1 83.811 Acc@5 99.295
[3,   100] loss: 0.449
[3,   200] loss: 0.470
[3,   300] loss: 0.451
epoch: 3 | Accuracy: 78.82590240624643
====> Acc@1 83.954 Acc@5 99.275
[4,   100] loss: 0.462
[4,   200] loss: 0.450
[4,   300] loss: 0.449
epoch: 4 | Accuracy: 79.75554880222984
====> Acc@1 84.029 Acc@5 99.281
[5,   100] loss: 0.418
[5,   200] loss: 0.470
[5,   300] loss: 0.445
epoch: 5 | Accuracy: 79.58348990248693
====> Acc@1 84.128 Acc@5 99.297
[6,   100] loss: 0.439
[6,   200] loss: 0.463
[6,   300] loss: 0.439
epoch: 6 | Accuracy: 82.3186963635535
====> Acc@1 84.165 Acc@5 99.311
[7,   100] loss: 0.437
[7,   200] loss: 0.455
[7,   300] loss: 0.435
epoch: 7 | Accuracy: 79.00470506874495
====> Acc@1 84.258 Acc@5 99.323
[8,   100] loss: 0.45

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for data in testLoader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Total accuracy (test images): {} %'.format(100*correct/total))

Total accuracy (test images): 91.14583333333333 %


In [ ]:
print(epochs_accuracy_list)

[]


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of airplane : 89 %
Accuracy of automobile : 92 %
Accuracy of  bird : 84 %
Accuracy of   cat : 82 %
Accuracy of  deer : 85 %
Accuracy of   dog : 87 %
Accuracy of  frog : 91 %
Accuracy of horse : 83 %
Accuracy of  ship : 93 %
Accuracy of truck : 92 %
